### Onur Can
#### 1. Importing & Pre-Processing of Data  &  2. Initialization of Data Structures
#### 3. Tree Inference & 4. Rule Extraction
#### 5. Training & Test Predictions and RMSE values & 6. Plotting Traning & Test & Fit ---------> ( P = 25 )
#### 7. Generalization to P = 5,10,15,.............50 and & 8. Final RMSE Plotting

In [4]:
# Onur Can 
# Project is done for Prof. Mehmet Gönen's DASC 521: Introduction to Machine Learning @ Koç University MSc Data Science Program
# Thanks Prof Mehmet for the dataset generation and instructions
import numpy as np
import matplotlib.pyplot as plt

### 1. Importing & Pre-Processing of Data

In [5]:
# importing the data set
data_set = np.genfromtxt("../input/volcan-dataset/volcano_data_set.csv", delimiter = "," , skip_header= 1)

# Splitting test-training data
training_set = data_set[0:150,:]
test_set = data_set[150:,:]
print(training_set.shape)
print(test_set.shape)

#Splitting and preparing the data
x_train = training_set[:,0]
y_train = training_set[:,1]
x_test = test_set[:,0]
y_test = test_set[:,1]

N_train = len(y_train)
N_test = len(y_test)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
print("min x_train value = ", np.min(x_train), "max x_train value = ", np.max(x_train))
print("min x_test value = ", np.min(x_test), "max x_test value = ", np.max(x_test))

### 2. Initialization of Data Structures

In [6]:
# create necessary data structures

P = 25 # Pre_Pruning Parameter / will be changed later at PART 6.

# This is for plotting purposes
lower_bound = +1.50
upper_bound = +5.2
lower_bound_interval = +1.5 # interval values will be our prediction boundaries
upper_bound_interval = +5.2
data_interval = np.linspace(lower_bound,upper_bound,371)

# Setting up Dictionaries for nodes data
node_indices = {}
is_terminal = {}
need_split = {}
node_splits = {}
node_frequencies = {}
# Below two dictionaries were added to to keep track of:
node_split_interval = {}    # after splits, keeping the track of split invervals to predict
node_averages = {}          # this is the prediction value if new_data in the interval

# put all training instances into the root node
node_indices[1] = np.array(range(N_train))
is_terminal[1] = False
need_split[1] = True
# These two dictionaries will be updated for each node see below
node_split_interval[1] = [lower_bound_interval, upper_bound_interval] # initialized via all data_interval
node_averages[1] = np.sum(y_train) / N_train # initiliazed via mean

### 3. Tree Inference

In [7]:
# learning algorithm
while True:
    # find nodes that need splitting
    split_nodes = [key for key, value in need_split.items() if value == True]
    # check whether we reach all terminal nodes
    # if this is 0 then there is no elements in need_split nodes array. stop algorithm
    # ie. no nodes requires more splitting
    if len(split_nodes) == 0:
        break
    
    # find best split positions for all nodes
    for split_node in split_nodes:
        data_indices = node_indices[split_node]
        # the node is either gonna be splitted or a gonna be a terminal node so need_split = False
        need_split[split_node] = False
        node_frequencies[split_node] = len(y_train[data_indices])
        
        # a node reaches element count < P it will automatically be a terminal node
        if len(y_train[data_indices]) <= P:
            is_terminal[split_node] = True
        # if not terminal condition satisfied then we will split in else statement
        else:
            is_terminal[split_node] = False
            
            # Preparation for split score calculations
            unique_values = np.sort(x_train[data_indices])
            # middle points of consecutive x_data points
            split_positions = (unique_values[1:len(unique_values)] + unique_values[0:(len(unique_values) - 1)]) / 2
            split_scores = np.repeat(0.0, len(split_positions))
            left_branch_averages = np.repeat(0.0, len(split_positions))
            right_branch_averages = np.repeat(0.0, len(split_positions))
            
            for s in range(len(split_positions)):
                left_indices = data_indices[x_train[data_indices] >= split_positions[s]]
                right_indices = data_indices[x_train[data_indices] <= split_positions[s]]
                # For each split calculate the -MEAN SQUARED ERROR-
                split_scores[s] = ( np.sum((y_train[left_indices] - np.mean(y_train[left_indices]))**2) + np.sum((
                y_train[right_indices] - np.mean(y_train[right_indices]))**2))  * 1 / len(unique_values)
                left_branch_averages[s] = np.mean(y_train[left_indices])
                right_branch_averages[s] = np.mean(y_train[right_indices])
                
            # Argmin for minimum error index
            best_splits = split_positions[np.argmin(split_scores)]
            # Node averages of Left and Right Child 
            left_branch_average = left_branch_averages[np.argmin(split_scores)]
            right_branch_average = right_branch_averages[np.argmin(split_scores)]

            # decided where to split along X axis
            # add it to dictinary with node_split key value
            node_splits[split_node] = best_splits
            
            # create left node using the selected split
            
            left_indices = data_indices[x_train[data_indices] >= best_splits]
            node_indices[2 * split_node] = left_indices
            is_terminal[2 * split_node] = False
            need_split[2 * split_node] = True
            #for any left child upper bound is coming from parent while lower is split_position
            node_split_interval[2 * split_node] = [best_splits, node_split_interval[split_node][1]]
            # Update child node averages
            node_averages[2 * split_node] = left_branch_average
      
            # create right node using the selected split
            right_indices = data_indices[x_train[data_indices] <= best_splits]
            node_indices[2 * split_node + 1] = right_indices
            is_terminal[2 * split_node + 1] = False
            need_split[2 * split_node + 1] = True
            #for any right child lower bound is coming from parent while upper is split_position
            node_split_interval[2 * split_node + 1] = [node_split_interval[split_node][0], best_splits]
            # Update child node averages
            node_averages[2 * split_node + 1] = right_branch_average
            
(print("--------------Which Nodes are Terminal -----------"))            
print(is_terminal)
(print("--------------Split Positions of Nodes -----------"))  
print(node_splits)
(print("--------------Node Frequencies -----------"))  
print(node_frequencies)
(print("--------------Split Interval of Each Node -----------")) 
print(node_split_interval)
(print("--------------Node Averages -----------")) 
print(node_averages)

### 4. Rule Extraction

In [8]:
# extract rules
print(" When P = ", P, " ----------------------------------------" )
terminal_nodes = [key for key, value in is_terminal.items() if value == True]
for terminal_node in terminal_nodes:
    index = terminal_node
    rules = np.array([])
    while index > 1:
        parent = np.floor(index / 2)
        if index % 2 == 0:
            # if node is left child of its parent
            rules = np.append(rules, "x > {:.2f}".format(node_splits[parent]))
        else:
            # if node is right child of its parent
            rules = np.append(rules, "x <= {:.2f}".format(node_splits[parent]))
        index = parent
    rules = np.flip(rules)
    print(" Terminal Node index {} Ruleset --- {} => {} ".format(terminal_node, rules, node_frequencies[terminal_node]))

### 5. Training & Test Predictions and RMSE values

In [9]:
# We have trained our model with traning data and our tree's splits and 
# interval for splits are determined. Put new x_i's to predict  

#traning set predictions
terminals =  [key for key, value in is_terminal.items() if value == True]
x_train_pred = np.empty(N_train)
# For each x_train data check which interval it fits in terminal nodes intervals dictionary
for i in range(len(x_train_pred)):
    for j in terminals:
        if node_split_interval[j][0] <= x_train[i] and x_train[i] < node_split_interval[j][1]:
            x_train_pred[i] = node_averages[j]

#test set predictions
x_test_pred = np.empty(N_test)
# For each x_test data check which interval it fits in terminal nodes intervals dictionary
for i in range(len(x_test_pred)):
    for j in terminals:
        if node_split_interval[j][0] <= x_test[i] and x_test[i] <= node_split_interval[j][1]:
            x_test_pred[i] = node_averages[j]

# For plotting purposes black line from +1.5 to +5.2 and predict each value
data_interval_pred = np.empty(len(data_interval))
for i in range(len(data_interval)):
    for j in terminals:
        if node_split_interval[j][0] <= data_interval[i] and data_interval[i] <= node_split_interval[j][1]:
            data_interval_pred[i] = node_averages[j]
            
RMSE_training = np.sqrt(np.sum(((x_train_pred - y_train)**2)) / N_train )
print("--------------------------------------------------------")
print("Training Set RMSE = ", RMSE_training," when P = ", P)
print("--------------------------------------------------------")
RMSE_test = np.sqrt(np.sum(((x_test_pred - y_test)**2)) / N_test )
print("--------------------------------------------------------")
print("Test Set RMSE = ", RMSE_test," when P = ", P)
print("--------------------------------------------------------")


#print(terminals)

### 6. Plotting Traning & Test & Fit ( P = 25 )

In [10]:
#Plotting all the fields in the homework
plt.figure( figsize = (12,6))
plt.plot(x_train, y_train, "b.", markersize = 10, label = "training")
plt.plot(x_test, y_test, "r.", markersize = 10, label = "test")
plt.plot(data_interval, data_interval_pred, "k-")
plt.grid()
plt.xlabel("Eruption Time (min)")
plt.ylabel("Waiting time to next eruption (min)")
plt.legend(loc="upper left")
plt.show()

### 7. Generalization to P = 5,10,15,.............50 and their RMSE Plotting
##### Defined function is exact copy of Part 3. It is for just to iterate over P values

In [11]:
# Set P in wanted range
# Define arrays for holding parameter values and their Respective RMSE_Train / Test Values
Prune_parameter = np.arange(5,55,5)
RMSE_training_values = np.empty(len(Prune_parameter))
RMSE_test_values = np.empty(len(Prune_parameter))

def RMSE_by_P( K , set1, set2):
    x_train_by_P = set1[:,0]
    y_train_by_P = set1[:,1]
    x_test_by_P = set2[:,0]
    y_test_by_P = set2[:,1]
    N_train_by_P = len(y_train_by_P)
    N_test_by_P = len(y_test_by_P)
    Prune = K
    lower_bound_interval_by_P = +1.5
    upper_bound_interval_by_P = +5.2
    node_indices_by_P = {}
    is_terminal_by_P = {}
    need_split_by_P = {}
    node_splits_by_P = {}
    node_frequencies_by_P = {}
    node_split_interval_by_P = {}    # after splits, keeping the track of split invervals to predict
    node_averages_by_P = {} 
    node_indices_by_P[1] = np.array(range(N_train_by_P))
    is_terminal_by_P[1] = False
    need_split_by_P[1] = True
    node_split_interval_by_P[1] = [lower_bound_interval_by_P, upper_bound_interval_by_P]
    node_averages_by_P[1] = np.sum(y_train_by_P) / N_train_by_P
    while True:
        split_nodes_by_P = [key for key, value in need_split_by_P.items() if value == True]
        if len(split_nodes_by_P) == 0:
            break
        for split_node_by_P in split_nodes_by_P:
            data_indices_by_P = node_indices_by_P[split_node_by_P]
            need_split_by_P[split_node_by_P] = False
            node_frequencies_by_P[split_node_by_P] = len(y_train_by_P[data_indices_by_P])
            if len(y_train_by_P[data_indices_by_P]) <= Prune:
                is_terminal_by_P[split_node_by_P] = True
            else:
                is_terminal_by_P[split_node_by_P] = False
                unique_values_by_P = np.sort(x_train_by_P[data_indices_by_P])
                split_positions_by_P = (unique_values_by_P[1:len(unique_values_by_P)] + unique_values_by_P[0:(len(unique_values_by_P) - 1)]) / 2
                split_scores_by_P = np.repeat(0.0, len(split_positions_by_P))
                left_branch_averages_by_P = np.repeat(0.0, len(split_positions_by_P))
                right_branch_averages_by_P = np.repeat(0.0, len(split_positions_by_P))
                for s in range(len(split_positions_by_P)):
                    left_indices_by_P = data_indices_by_P[x_train_by_P[data_indices_by_P] >= split_positions_by_P[s]]
                    right_indices_by_P = data_indices_by_P[x_train_by_P[data_indices_by_P] <= split_positions_by_P[s]]
                    split_scores_by_P[s] = ( np.sum((y_train_by_P[left_indices_by_P] - np.mean(y_train_by_P[left_indices_by_P]))**2) + np.sum((
                    y_train_by_P[right_indices_by_P] - np.mean(y_train_by_P[right_indices_by_P]))**2))  * 1 / len(unique_values_by_P)
                    left_branch_averages_by_P[s] = np.mean(y_train_by_P[left_indices_by_P])
                    right_branch_averages_by_P[s] = np.mean(y_train_by_P[right_indices_by_P])
                best_splits_by_P = split_positions_by_P[np.argmin(split_scores_by_P)]
                left_branch_average_by_P = left_branch_averages_by_P[np.argmin(split_scores_by_P)]
                right_branch_average_by_P = right_branch_averages_by_P[np.argmin(split_scores_by_P)]
                node_splits[split_node_by_P] = best_splits_by_P
                left_indices_by_P = data_indices_by_P[x_train_by_P[data_indices_by_P] >= best_splits_by_P]
                node_indices_by_P[2 * split_node_by_P] = left_indices_by_P
                is_terminal_by_P[2 * split_node_by_P] = False
                need_split_by_P[2 * split_node_by_P] = True
                node_split_interval_by_P[2 * split_node_by_P] = [best_splits_by_P, node_split_interval_by_P[split_node_by_P][1]]
                node_averages_by_P[2 * split_node_by_P] = left_branch_average_by_P
                right_indices_by_P = data_indices_by_P[x_train_by_P[data_indices_by_P] <= best_splits_by_P]
                node_indices_by_P[2 * split_node_by_P + 1] = right_indices_by_P
                is_terminal_by_P[2 * split_node_by_P + 1] = False
                need_split_by_P[2 * split_node_by_P + 1] = True
                node_split_interval_by_P[2 * split_node_by_P + 1] = [node_split_interval_by_P[split_node_by_P][0], best_splits_by_P]
                node_averages_by_P[2 * split_node_by_P + 1] = right_branch_average_by_P   
    terminals_by_P =  [key for key, value in is_terminal_by_P.items() if value == True]
    x_train_pred_by_P = np.empty(N_train_by_P)
    for i in range(len(x_train_pred_by_P)):
        for j in terminals_by_P:
            if node_split_interval_by_P[j][0] <= x_train_by_P[i] and x_train_by_P[i] < node_split_interval_by_P[j][1]:
                x_train_pred_by_P[i] = node_averages_by_P[j]
    x_test_pred_by_P = np.empty(N_test_by_P)
    for i in range(len(x_test_pred_by_P)):
        for j in terminals_by_P:
            if node_split_interval_by_P[j][0] <= x_test_by_P[i] and x_test_by_P[i] <= node_split_interval_by_P[j][1]:
                x_test_pred_by_P[i] = node_averages_by_P[j]
    return x_train_pred_by_P, x_test_pred_by_P

#Iterating over P values while calling the function
for i in range(len(Prune_parameter)):
    x_train_pred_by_P, x_test_pred_by_P = RMSE_by_P(Prune_parameter[i], training_set, test_set)
    RMSE_training_values[i] = np.sqrt(np.sum(((x_train_pred_by_P - y_train)**2)) / N_train )
    RMSE_test_values[i] =  np.sqrt(np.sum(((x_test_pred_by_P - y_test)**2)) / N_test )
    print("When P = ", Prune_parameter[i], "--- RMSE_train = ", RMSE_training_values[i] , "--- RMSE_test = ", RMSE_test_values[i]) 

### 8. Final RMSE Plotting

In [12]:
#Putting all arrays together
plt.figure(figsize = (8,8))
plt.plot(Prune_parameter, RMSE_training_values, "b.-", label = "training", markersize= 12)
plt.plot(Prune_parameter, RMSE_test_values, "r.-", label = "test", markersize= 12)
plt.xlabel("Pre-pruning size(P)")
plt.ylabel("RMSE")
plt.legend(loc="upper right")
plt.show()